## Use Pretrained model to improve accuracy

In [30]:
from keras.datasets import cifar10
from keras.models import Sequential,load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten,Dropout,BatchNormalization,GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.applications import VGG16,InceptionV3,MobileNet,VGG19
from keras.models import Model
import scipy as sp
import numpy as np
from PIL import Image
epochs = 20
batch_size = 64
num_classes = 10
input_size = (64,64)

In [2]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [3]:
X_train,X_val,Y_train,Y_val = train_test_split(x_train,y_train,test_size=0.2,random_state=11)

In [4]:
Y_train = to_categorical(Y_train,num_classes)
Y_test = to_categorical(y_test,num_classes)
Y_val = to_categorical(Y_val,num_classes)

In [5]:
X_train = np.array([np.array(Image.fromarray(x).resize((64,64))) for x in X_train])
X_val = np.array([np.array(Image.fromarray(x).resize((64,64))) for x in X_val])
X_test = np.array([np.array(Image.fromarray(x).resize((64,64))) for x in x_test])

In [7]:
base = VGG16(weights='imagenet',
                    include_top=False,
                    input_shape = (64,64,3))
'''base = MobileNet(weights = 'imagenet',
                    include_top = False,
                    input_shape = (64,64,3))'''

"base = MobileNet(weights = 'imagenet',\n                    include_top = False,\n                    input_shape = (64,64,3))"

In [12]:
base.summary()
#base.layers.pop()
inputs = base.layers[-6].output

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0     

In [13]:
## fine tuning
'''for layer in base.layers:
    if layer.name == 'block5_conv1':
        layer.tranable = True
    else:
        layer.trainable = False'''
for layer in base.layers:
    layer.trainable= False

In [62]:
'''x = Flatten()(inputs)
x = BatchNormalization()(x)
x = Dense(256,activation='relu')(x)'''
x = Conv2D(512,(3,3),activation='relu')(inputs)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.8)(x)
x = BatchNormalization()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.8)(x)
x = BatchNormalization()(x)
x = Dense(num_classes,activation='softmax')(x)
model = Model(base.input,x)

In [63]:
len(model.trainable_weights)

14

In [64]:
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0   

In [65]:
model.compile(loss='categorical_crossentropy',
             optimizer = 'adam',
             metrics=['acc'],
             )

In [66]:
model_save_path = "val_loss-{val_loss:.4f}-val_acc{val_acc:4f}.hdf5"
callbacks_list = [
    EarlyStopping(
        monitor='val_acc',
        patience=7
    ),
    ModelCheckpoint(
        model_save_path,
        monitor='val_acc',
        save_best_only = True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=5
    )
    
]

In [67]:
history = model.fit(X_train,Y_train,
         batch_size = batch_size,
         epochs = epochs,
         validation_data = (X_val,Y_val),
         callbacks = callbacks_list)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 13s 325us/step - loss: 1.2533 - acc: 0.5701 - val_loss: 0.5759 - val_acc: 0.8038
Epoch 2/20
40000/40000 [==============================] - 12s 309us/step - loss: 0.7162 - acc: 0.7654 - val_loss: 0.4767 - val_acc: 0.8362
Epoch 3/20
40000/40000 [==============================] - 12s 307us/step - loss: 0.5682 - acc: 0.8162 - val_loss: 0.4481 - val_acc: 0.8483
Epoch 4/20
40000/40000 [==============================] - 12s 310us/step - loss: 0.4779 - acc: 0.8445 - val_loss: 0.4173 - val_acc: 0.8615
Epoch 5/20
40000/40000 [==============================] - 12s 310us/step - loss: 0.4137 - acc: 0.8655 - val_loss: 0.3993 - val_acc: 0.8643
Epoch 6/20
40000/40000 [==============================] - 12s 310us/step - loss: 0.3557 - acc: 0.8860 - val_loss: 0.4250 - val_acc: 0.8559
Epoch 7/20
40000/40000 [==============================] - 12s 311us/step - loss: 0.3031 - acc: 0.9037 - val_loss: 0.

In [68]:
tt = load_model('val_loss-0.3878-val_acc0.880500.hdf5')
scores = tt.evaluate(X_test,Y_test,verbose=1)
scores

10000/10000 [==============================] - 2s 249us/step


[0.3911371708214283, 0.8863999843597412]

## VGG16 = 0.823, img_size = 48,48
## VGG16 = 0.854, img_size = 64,64
## VGG16 = 0.882, img_size = 64,64 --> with GAP


## Done###

* 기존 vgg모델의 출력층을 몇개씩 빼보면서 실험을 진행해봄 --> -2일때도 한번 해보자
* 바닥부터 내맘대로 모델 만들어서도 시험해봄(정확도 대략 70?)
* 70 --> 88까지 올림 kaggle 1등이 95니깐 가즈앗
* train data 수가 부족하기때문에 Data augmentation 적용해보자 --> 잘 안오른다..

## Todo ###

In [18]:
predictions = tt.predict(X_test,verbose=1)

10000/10000 [==============================] - 2s 184us/step


In [21]:
predicted_classes = np.argmax(predictions,axis=1)
predicted_classes

array([3, 8, 8, ..., 5, 1, 7])

In [36]:
from sklearn.metrics import classification_report
print(classification_report(yy,predicted_classes, target_names=['class 0', 'class 1','class 2', 'class 3','class 4', 'class 5','class 6', 'class 7','class 8', 'class 9']))


              precision    recall  f1-score   support

     class 0       0.89      0.92      0.91      1000
     class 1       0.96      0.92      0.94      1000
     class 2       0.92      0.81      0.86      1000
     class 3       0.78      0.73      0.75      1000
     class 4       0.88      0.87      0.88      1000
     class 5       0.78      0.81      0.79      1000
     class 6       0.90      0.93      0.92      1000
     class 7       0.90      0.93      0.92      1000
     class 8       0.92      0.96      0.94      1000
     class 9       0.91      0.94      0.93      1000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

